# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
from api_keys import g_key

gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvpath = os.path.join('weather.csv')
cities_data = pd.read_csv(csvpath)
cities_data

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Burgos,42.3394,-3.7079,13.23,82,0,1.03,ES,1627087415
1,1,Breznik,42.7414,22.8981,16.33,77,1,0.32,BG,1627087238
2,2,Sa Kaeo,13.7500,102.3333,23.64,96,100,2.69,TH,1627087264
3,3,Citrus,34.1150,-117.8917,38.80,44,1,2.24,US,1627087415
4,4,Milicz,51.5277,17.2714,16.59,58,91,1.34,PL,1627087415
...,...,...,...,...,...,...,...,...,...,...
474,474,Borjomi,41.8527,43.4128,16.76,94,99,0.67,GE,1627087540
475,475,Le Portel,50.7056,1.5757,19.27,78,27,7.66,FR,1627087540
476,476,Šardice,48.9640,17.0281,18.18,80,0,1.60,CZ,1627087540
477,477,Prainha,-1.8000,-53.4800,26.01,63,98,1.65,BR,1627087541


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = cities_data[["Latitude", "Longitude"]]
humidity = cities_data["Humidity"].astype(float)


In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
cities_data_w1 = cities_data.loc[cities_data["Max Temp"] < 25]
cities_data_w2 = cities_data_w1.loc[cities_data_w1["Cloudiness"] == 0]
cities_data_w3 = cities_data_w2.loc[cities_data_w2["Max Temp"] > 20]
hotel_df = cities_data_w3.loc[cities_data_w3["Wind Speed"] < 3]
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,Delvinë,39.9511,20.0978,23.41,54,0,2.37,AL,1627087422
32,32,Bollène,44.2848,4.7530,24.93,50,0,2.57,FR,1627087423
50,50,Santa Helena de Goiás,-17.8136,-50.5969,20.79,45,0,2.70,BR,1627087427
97,97,Itapuí,-22.2333,-48.7192,22.00,42,0,1.72,BR,1627087440
104,104,Bāneh,35.9975,45.8853,20.85,41,0,0.95,IR,1627087442
150,150,Sertãozinho,-21.1378,-47.9903,21.06,44,0,2.35,BR,1627087453
165,165,Arys,42.4252,68.7950,23.12,20,0,1.87,KZ,1627087457
168,168,Le Grand-Quevilly,49.4008,1.0458,21.58,72,0,2.06,FR,1627087459
187,187,Santa Marinella,42.0345,11.8536,24.75,73,0,1.97,IT,1627087464
217,217,Martino,38.5667,23.2167,23.03,47,0,1.11,GR,1627087472


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-34-56b4fe6e59b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
30,30,Delvinë,39.9511,20.0978,23.41,54,0,2.37,AL,1627087422,
32,32,Bollène,44.2848,4.7530,24.93,50,0,2.57,FR,1627087423,
50,50,Santa Helena de Goiás,-17.8136,-50.5969,20.79,45,0,2.70,BR,1627087427,
97,97,Itapuí,-22.2333,-48.7192,22.00,42,0,1.72,BR,1627087440,
104,104,Bāneh,35.9975,45.8853,20.85,41,0,0.95,IR,1627087442,
150,150,Sertãozinho,-21.1378,-47.9903,21.06,44,0,2.35,BR,1627087453,
165,165,Arys,42.4252,68.7950,23.12,20,0,1.87,KZ,1627087457,
168,168,Le Grand-Quevilly,49.4008,1.0458,21.58,72,0,2.06,FR,1627087459,
187,187,Santa Marinella,42.0345,11.8536,24.75,73,0,1.97,IT,1627087464,
217,217,Martino,38.5667,23.2167,23.03,47,0,1.11,GR,1627087472,


In [35]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Missing field/result... skipping.


C:\Users\Tako\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [42]:
hotel_df
hotel_df2 = hotel_df.dropna()
hotel_df2

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
32,32,Bollène,44.2848,4.7530,24.93,50,0,2.57,FR,1627087423,Logis Hôtel la Bastide St Bach
50,50,Santa Helena de Goiás,-17.8136,-50.5969,20.79,45,0,2.70,BR,1627087427,Castro Castle Hotel
104,104,Bāneh,35.9975,45.8853,20.85,41,0,0.95,IR,1627087442,Behzad Hotel
150,150,Sertãozinho,-21.1378,-47.9903,21.06,44,0,2.35,BR,1627087453,Comfort Hotel Sertãozinho
165,165,Arys,42.4252,68.7950,23.12,20,0,1.87,KZ,1627087457,"Gostinitsa ""Madani"""
168,168,Le Grand-Quevilly,49.4008,1.0458,21.58,72,0,2.06,FR,1627087459,"Radisson Blu Hotel, Rouen Centre"
187,187,Santa Marinella,42.0345,11.8536,24.75,73,0,1.97,IT,1627087464,Hotel Borgo del Mare
217,217,Martino,38.5667,23.2167,23.03,47,0,1.11,GR,1627087472,Eftopia Martino Pansion
235,235,Reisterstown,39.4696,-76.8294,24.71,68,0,1.52,US,1627087449,Holiday Inn Express & Suites Owings Mills-Balt...
261,261,Argos,37.6333,22.7333,23.18,47,0,1.86,GR,1627087483,Alexandros Boutique Hotel


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_2 = hotel_df[["Latitude", "Longitude"]]

In [49]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations_2)
fig.add_layer(markers)
plt.savefig("Vacation spots.png")
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>